# Test

At vero eos et accusamus et iusto odio dignissimos ducimus qui blanditiis praesentium.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
  .master('yarn') \
  .appName('spark-bigquery-crypto') \
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .config("spark.jars.packages", "graphframes:graphframes:0.8.1-spark2.4-s_2.12") \
  .getOrCreate()

In [3]:
bucket = "dataproc-staging-us-central1-397709471406-lrrymuq9"
spark.conf.set('temporaryGcsBucket', bucket)

In [4]:
# Load data from BigQuery.
table = 'bigquery-public-data:crypto_ethereum.transactions'

eth_transactions = spark.read \
  .format('bigquery') \
  .option('table', table) \
  .option("filter", "DATE(block_timestamp) between '2020-01-01' and '2020-01-31'") \
  .load()

# eth_transactions = spark.read.format('bigquery') \
#   .option('table', 'charged-sector-315517:ethereum_onchain.transactions') \
#   .load()

eth_transactions.createOrReplaceTempView('eth_transactions')

# Perform SQL query.
eth_data = spark.sql(
    '''SELECT * FROM eth_transactions limit 10''')
eth_data.show()

In [5]:
# Perform SQL query.
eth_data = spark.sql(
    '''SELECT * FROM eth_transactions limit 10''')
eth_data.show()
# eth_data.printSchema()

+-------------------------------+
|count(DISTINCT block_timestamp)|
+-------------------------------+
|                          10348|
+-------------------------------+

root
 |-- count(DISTINCT block_timestamp): long (nullable = false)



In [6]:
# Perform SQL query.
eth_data = spark.sql(
    '''SELECT * FROM eth_transactions WHERE DATE(block_timestamp) between "2019-01-01" and "2019-01-31"''')
# eth_data.show()
# eth_data.printSchema()

In [ ]:
eth_data.write.format('bigquery') \
  .option('table', 'crypto_ethereum.transactions') \
  .save()

Py4JJavaError: An error occurred while calling o73.save.
: java.lang.RuntimeException: Failed to write to BigQuery
	at com.google.cloud.spark.bigquery.BigQueryWriteHelper.writeDataFrameToBigQuery(BigQueryWriteHelper.scala:94)
	at com.google.cloud.spark.bigquery.BigQueryInsertableRelation.insert(BigQueryInsertableRelation.scala:43)
	at com.google.cloud.spark.bigquery.BigQueryRelationProvider.createRelation(BigQueryRelationProvider.scala:116)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:46)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:86)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$ex

In [ ]:
df = spark.read \
  .format('bigquery') \
  .option('table', 'crypto_ethereum.transactions') \
  .load()

df.printSchema()